In [192]:
import numpy as np
import seaborn as sns
import pandas as pd
import os
from inspect import getmembers
import matplotlib.pyplot as plt
import networkx as nx
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import dill
from equilibrator_api import ComponentContribution, Q_, Reaction
import requests
import xmltodict
import pint
import cvxpy as cp
import itertools

sns.set(style='darkgrid', palette='viridis', context='talk')

os.chdir(os.path.expanduser('~/vivarium-ecoli'))

A + B -E1> C -E2> D

B -E3> F

A -E4> G

# Testing eQuilibrator

In [2]:
cc = ComponentContribution()

# optional: changing the aqueous environment parameters
cc.p_h = Q_(7.4)
cc.p_mg = Q_(3.0)
cc.ionic_strength = Q_("0.25M")
cc.temperature = Q_("298.15K")



In [3]:
from equilibrator_api import Reaction
compound_ids = ["WATER", "ADP", "ATP", "Pi"]
compound_dict = {cid : cc.get_compound(f"metacyc.compound:{cid}") for cid in compound_ids}
atpase_reaction = Reaction({
    compound_dict["ATP"]: -1,
    compound_dict["WATER"]: -1,
    compound_dict["ADP"]: 1,
    compound_dict["Pi"]: 1,
})

In [4]:
standard_dg_prime = cc.standard_dg_prime(atpase_reaction)
standard_dg_prime

<Measurement(-29.144472212600157, 0.3042778553577678, kilojoule / mole)>

In [5]:
cytoplasmic_p_h = Q_(7.5)
cytoplasmic_ionic_strength = Q_("250 mM")
periplasmic_p_h = Q_(7.0)
periplasmic_ionic_strength = Q_("200 mM")
e_potential_difference = Q_("0.15 V")
cytoplasmic_reaction = "bigg.metabolite:pep = bigg.metabolite:g6p + bigg.metabolite:pyr"
periplasmic_reaction = "bigg.metabolite:glc__D = "

cc = ComponentContribution()
cc.p_h = cytoplasmic_p_h
cc.ionic_strength = cytoplasmic_ionic_strength
standard_dg_prime = cc.multicompartmental_standard_dg_prime(
    cc.parse_reaction_formula(cytoplasmic_reaction),
    cc.parse_reaction_formula(periplasmic_reaction),
    e_potential_difference=e_potential_difference,
    p_h_outer=periplasmic_p_h,
    ionic_strength_outer=periplasmic_ionic_strength,
)

print(standard_dg_prime)


(-44.8 +/- 0.6) kilojoule / mole


In [6]:
cc.get_compound_by_inchi("WQZGKKKJIJFFOK-GASJEMHNSA-N")

# Toy example with fake data

In [7]:
s = requests.Session() # create session
# Post login credentials to session:
s.post('https://websvc.biocyc.org/credentials/login/', data={'email':'cellulararchitect@protonmail.com', 'password':'Cellman0451'})
# Issue web service request:
r = s.get('https://websvc.biocyc.org/getxml?id=ECOLI:6PFRUCTPHOS-RXN&detail=low&fmt=json')

In [19]:
rxn = xmltodict.parse(r.content)
rxn['ptools-xml']['Reaction']['right']

[{'Compound': {'@resource': 'getxml?ECOLI:PROTON',
   '@orgid': 'ECOLI',
   '@frameid': 'PROTON'}},
 {'Compound': {'@resource': 'getxml?ECOLI:ADP',
   '@orgid': 'ECOLI',
   '@frameid': 'ADP'}},
 {'Compound': {'@resource': 'getxml?ECOLI:FRUCTOSE-16-DIPHOSPHATE',
   '@orgid': 'ECOLI',
   '@frameid': 'FRUCTOSE-16-DIPHOSPHATE'}}]

In [25]:
rxn['ptools-xml']['Reaction']['left'][0]['Compound']['@frameid']

'FRUCTOSE-6P'

In [52]:
r = s.get(f'https://websvc.biocyc.org/getxml?id=ECOLI:F16ALDOLASE-RXN&detail=low&fmt=json')
rxn = xmltodict.parse(r.content)
rxn['ptools-xml']['Reaction']['left']

{'Compound': {'@resource': 'getxml?ECOLI:FRUCTOSE-16-DIPHOSPHATE',
  '@orgid': 'ECOLI',
  '@frameid': 'FRUCTOSE-16-DIPHOSPHATE'}}

In [90]:
rxns_names = ['6PFRUCTPHOS-RXN', 'F16ALDOLASE-RXN', '2TRANSKETO-RXN', 'TRIOSEPISOMERIZATION-RXN']

rxns_dict = {}
stoich_dict = {}

for name in rxns_names:
    r = s.get(f'https://websvc.biocyc.org/getxml?id=ECOLI:{name}&detail=low&fmt=json')
    rxn = xmltodict.parse(r.content)

    rxn_dict = {}
    stoich_loop_dict = {}
    left = rxn['ptools-xml']['Reaction']['left']
    right = rxn['ptools-xml']['Reaction']['right']
    
    if type(left) is dict:
        left = [left]
    
    if type(right) is dict:
        right = [right]
    
    for mol in left:
        if type(mol) is dict:
            cid = mol['Compound']['@frameid']
            mol_cc = cc.get_compound(f"metacyc.compound:{cid}")
            rxn_dict[mol_cc] = -1
            stoich_loop_dict[cid] = -1

    for mol in right:
        if type(mol) is dict:
            cid = mol['Compound']['@frameid']
            mol_cc = cc.get_compound(f"metacyc.compound:{cid}")
            rxn_dict[mol_cc] =  1
            stoich_loop_dict[cid] = 1
    
    rxns_dict[name] = Reaction(rxn_dict)
    stoich_dict[name] = stoich_loop_dict
    
rxns_dict

{'6PFRUCTPHOS-RXN': <equilibrator_api.phased_reaction.PhasedReaction at 0x17fbcf3d0>,
 'F16ALDOLASE-RXN': <equilibrator_api.phased_reaction.PhasedReaction at 0x2876a6c70>,
 '2TRANSKETO-RXN': <equilibrator_api.phased_reaction.PhasedReaction at 0x28918cee0>,
 'TRIOSEPISOMERIZATION-RXN': <equilibrator_api.phased_reaction.PhasedReaction at 0x107ad9df0>}

In [56]:
list(rxns_dict.values())

In [57]:
(standard_dg_prime, dg_uncertainty) = cc.standard_dg_prime_multi(list(rxns_dict.values()), uncertainty_representation="cov")

In [58]:
standard_dg_prime

Magnitude,[-21.05010908198642 22.047924530096907 -5.946115510564027 -5.617742386692498]
Units,kilojoule/mole


In [59]:
dg_uncertainty

Magnitude,[[1.5398212990074922 -2.2167471180569773 0.6541085596545129 -0.004664260811860303] [-2.2167471180569773 5.286946034707553 -2.4322877177623594 -0.1291182736017488] [0.6541085596545129 -2.4322877177623594 5.547270812427002 -0.08893998540652082] [-0.004664260811860303 -0.1291182736017488 -0.08893998540652082 0.3010346102557717]]
Units,kilojoule2/mole2


In [61]:
R = 0.008314 # kJ/mol*K
T = 298.15 # K

In [75]:
dG = standard_dg_prime._magnitude

keq = np.exp(-dG/(R*T))
keq

array([4.87556287e+03, 1.37137368e-04, 1.10096201e+01, 9.64363243e+00])

In [76]:
vars(standard_dg_prime)

{'_magnitude': array([-21.05010908,  22.04792453,  -5.94611551,  -5.61774239]),
 '_units': <UnitsContainer({'kilojoule': 1, 'mole': -1})>,
 '_Quantity__used': False,
 '_Quantity__handling': None,
 '_dimensionality': <UnitsContainer({'[length]': 2, '[mass]': 1, '[substance]': -1, '[time]': -2})>}

# Set up problem

Stochiometric matrix:

In [234]:
Sd = pd.DataFrame(stoich_dict, dtype=np.int8).fillna(0).astype(np.int8)

n_met = len(Sd.index)
n_rxn = len(Sd.columns)

Sd

,6PFRUCTPHOS-RXN,F16ALDOLASE-RXN,2TRANSKETO-RXN,TRIOSEPISOMERIZATION-RXN
FRUCTOSE-6P,-1,0,1,0
ATP,-1,0,0,0
PROTON,1,0,0,0
ADP,1,0,0,0
FRUCTOSE-16-DIPHOSPHATE,1,-1,0,0
DIHYDROXY-ACETONE-PHOSPHATE,0,1,0,1
GAP,0,1,1,-1
ERYTHROSE-4P,0,0,-1,0
XYLULOSE-5-PHOSPHATE,0,0,-1,0


In [210]:
S = np.array(Sd)
S_s = np.copy(S)
S_p = -np.copy(S) #reverse neg sign
S_s[S < 0] = 0
S_p[S > 0] = 0

S_s_nz = np.array(S_s.nonzero())
S_p_nz = np.array(S_p.nonzero())

c_s_nz = S_s_nz[0, :]
c_p_nz = S_p_nz[0, :]

n_Km_s = np.max(S_s_nz.shape) 
n_Km_p = np.max(S_p_nz.shape)

In [211]:
c = cp.Variable(n_met)
Km_s = cp.Variable(n_Km_s)
Km_p = cp.Variable(n_Km_p)

In [257]:
c_s_nz

array([0, 2, 3, 4, 5, 5, 6, 6])

In [212]:
y_s = c[c_s_nz] - Km_s
y_p = c[c_p_nz] - Km_p

In [267]:
y_s

Expression(AFFINE, UNKNOWN, (8,))

In [213]:
sat_perm = np.array(list(itertools.product([0, 1], repeat=2)))
sat_perm[1:, :]

array([[0, 1],
       [1, 0],
       [1, 1]])

In [216]:
n_alpha = np.sum(np.power(2, S_s.sum(axis=0)) - 1)
n_beta = np.sum(np.power(2, S_p.sum(axis=0)) - 1)

In [268]:
met_s_nz

array([0, 2, 3, 4, 5, 5, 6, 6])

In [269]:
rxn_s_nz

array([2, 0, 0, 0, 1, 3, 1, 2])

In [275]:
len(idx_cur_rxn)

8

In [279]:
met_s_nz = S_s_nz[0, :]
rxn_s_nz = S_s_nz[1, :]    
met_p_nz = S_p_nz[0, :]
rxn_p_nz = S_p_nz[1, :]

C_alpha = np.zeros([n_alpha, len(met_s_nz)])
C_beta = np.zeros([n_beta, len(met_p_nz)])

D_alpha = np.zeros([n_rxn, n_alpha])
D_beta = np.zeros([n_rxn, n_beta])


idx = 0

for i in range(n_rxn):
    idx_cur_rxn = rxn_s_nz == i
    
    sat_perm = np.array(list(itertools.product([0, 1], repeat=sum(idx_cur_rxn))))
    sat_perm = sat_perm[1:, :]
    
    r, c = sat_perm.shape
    
    C_alpha[idx:(idx+r), idx_cur_rxn] = sat_perm
    #D_alpha[i, idx:(idx+r)] = 1
        
    idx += r # add row # 

idx = 0
    
for i in range(n_rxn):
    idx_cur_rxn = rxn_p_nz == i
    
    sat_perm = np.array(list(itertools.product([0, 1], repeat=sum(idx_cur_rxn))))
    sat_perm = sat_perm[1:, :]
    
    r, c = sat_perm.shape
    
    C_beta[idx:(idx+r), idx_cur_rxn] = sat_perm
    #D_alpha[i, idx:(idx+r)] = 1
        
    idx += r # add row # 

In [281]:
C_alpha @ y_s
C_beta @ y_p

Expression(AFFINE, UNKNOWN, (8,))

In [ ]:
idx = 0

for i in range(n_rxn):
    p_cur_rxn = met_p_nz[rxn_p_nz == i]
    
    sat_perm = np.array(list(itertools.product([0, 1], repeat=len(p_cur_rxn))))
    sat_perm = sat_perm[1:, :]
    
    r, c = sat_perm.shape
    
    C_beta[idx:(idx+r), p_cur_rxn] = sat_perm
    #D_beta[i, idx:(idx+r)] = 1
        
    idx += r # add row # 

    
# remove zero cols
#C_alpha = C_alpha[:, list(set(S_s_nz[0, :]))]

In [271]:
C_alpha

array([[0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0.]])

In [263]:
list(set(S_s_nz[0, :]))

[0, 2, 3, 4, 5, 6]

In [248]:
D_alpha

array([[1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [244]:
C_beta

array([[0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 1.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0.]])

In [221]:
S_s_nz

array([[0, 2, 3, 4, 5, 5, 6, 6],
       [2, 0, 0, 0, 1, 3, 1, 2]])

In [218]:
C_alpha = np.zeros([n_alpha, n_met])
C_beta = np.zeros([n_beta, n_met])

In [219]:
C_alpha

array([[0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [ ]:
C_alpha = np.zeros([])

In [171]:
c[c_nz] - Km

Expression(AFFINE, UNKNOWN, (14,))

In [155]:
c = cp.Variable(n_met)

expr = cp.multiply(cp.vstack([c,c,c,c]), Sm.T)

In [140]:
np.array(c)

array(Variable((9,)), dtype=object)

In [147]:
c[:, None]

Expression(AFFINE, UNKNOWN, (9, 1))

In [151]:
cp.multiply(cp.vstack([c,c,c,c]), Sm.T)

Expression(AFFINE, UNKNOWN, (4, 9))

# Import sim output

In [20]:
time = '10'
date = '2023-02-13'
experiment = 'balance'
entry = f'{experiment}_{time}_{date}'
folder = f'out/fbagd/{entry}/'

In [21]:
output = np.load(folder + 'output.npy',allow_pickle='TRUE').item()
# output = np.load(r"out/geneRxnVerifData/output_glc.npy", allow_pickle=True, encoding='ASCII').tolist()
output = output['agents']['0']
fba = output['listeners']['fba_results']
mass = output['listeners']['mass']
bulk = pd.DataFrame(output['bulk'])

In [22]:
f = open(folder + 'agent_processes.pkl', 'rb')
agent_processes = dill.load(f)
f.close()

f = open(folder + 'agent_steps.pkl', 'rb')
agent_steps = dill.load(f)
f.close()

In [23]:
getattributes(agent_steps['ecoli-metabolism'])

NameError: name 'getattributes' is not defined

In [24]:
s_matrix = agent_steps['ecoli-metabolism'].model.network.s_matrix
stoichiometry = agent_steps['ecoli-metabolism'].stoichiometry

AttributeError: 'FluxBalanceAnalysisModel' object has no attribute 'network'

In [ ]:
getmembers(agent_steps['ecoli-metabolism'])[7][1].keys()

In [ ]:
agent_processes

# Check FBA output for central glycolysis

In [10]:
stoichiometry

NameError: name 'stoichiometry' is not defined